# SVI Volatility Surface Calibration Demo

This notebook demonstrates how to use the `implied` Rust library with Python bindings to calibrate an SVI (Stochastic Volatility Inspired) volatility surface from a set of option prices.

## 1. Installation

This notebook assumes the `implied` library has been installed from the wheel. We also need to install `pandas`, `numpy`, and `plotly` for data manipulation and visualization.

In [1]:
!pip install pandas numpy plotly scipy


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


## 2. Imports

Now, let's import the necessary libraries.

In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from implied import OptionData, OptionType, SVIVolatilitySurface, SVIParameters, black_scholes_price_py, implied_volatility_py, CalibrationError

## 3. Data Generation

To test the calibration, we will generate a realistic set of option prices. We'll create a synthetic market where the "true" volatility smile is governed by a known set of SVI parameters for two different expiries (1 year and 2 years).

In [3]:
def generate_realistic_options(num_options, expiry, true_svi):
    spot = 100.0
    risk_free_rate = 0.05
    options = []
    for _ in range(num_options):
        strike = np.random.uniform(80, 120)
        
        # Create a temporary OptionData to calculate moneyness
        temp_opt = OptionData(strike=strike, spot=spot, expiry=expiry, price=0, risk_free_rate=risk_free_rate, option_type=OptionType.Call)
        moneyness = temp_opt.moneyness()
        
        total_variance = true_svi.a + true_svi.b * (true_svi.rho * (moneyness - true_svi.m) + np.sqrt((moneyness - true_svi.m)**2 + true_svi.sigma**2))
        if total_variance < 0:
            continue
        
        vol = np.sqrt(total_variance / expiry)
        price = black_scholes_price_py(spot, strike, risk_free_rate, expiry, vol, OptionType.Call)
        
        options.append(OptionData(
            strike=strike, 
            spot=spot, 
            expiry=expiry, 
            price=price, 
            risk_free_rate=risk_free_rate, 
            option_type=OptionType.Call
        ))
    return options

# Define the "true" SVI parameters for our synthetic market
true_svi_1y = SVIParameters(a=0.04, b=0.4, rho=-0.7, m=0.1, sigma=0.2)
true_svi_2y = SVIParameters(a=0.035, b=0.35, rho=-0.65, m=0.12, sigma=0.22)

# Generate options for two expiries
options_1y = generate_realistic_options(50, 1.0, true_svi_1y)
options_2y = generate_realistic_options(50, 2.0, true_svi_2y)
all_options = options_1y + options_2y

print(f"Generated {len(all_options)} options.")

Generated 100 options.


## 4. Calibrate the SVI Surface

Now, we pass the list of `OptionData` objects to the `SVIVolatilitySurface.calibrate` method. The Rust backend will group the options by expiry, calibrate a separate SVI slice for each, and return a surface object.

In [4]:
try:
    surface = SVIVolatilitySurface.calibrate(all_options)
    print("SVI surface calibrated successfully!")
except CalibrationError as e:
    print(f"Calibration failed: {e}")

SVI surface calibrated successfully!


## 5. Query the Volatility Surface

We can now use the `volatility` method on the calibrated surface to get the implied volatility for any moneyness and expiry. The library will find the closest calibrated expiry slice to the one requested.

In [5]:
# Exact expiry match
vol_1y = surface.volatility(moneyness=0.05, expiry=1.0)
print(f"Volatility at moneyness=0.05, expiry=1.0y: {vol_1y:.4f}")

# Closest expiry match
vol_1_1y = surface.volatility(moneyness=0.05, expiry=1.1) # Should use the 1.0y slice
print(f"Volatility at moneyness=0.05, expiry=1.1y: {vol_1_1y:.4f}")

vol_1_8y = surface.volatility(moneyness=0.05, expiry=1.8) # Should use the 2.0y slice
print(f"Volatility at moneyness=0.05, expiry=1.8y: {vol_1_8y:.4f}")

Volatility at moneyness=0.05, expiry=1.0y: 0.3694
Volatility at moneyness=0.05, expiry=1.1y: 0.3694
Volatility at moneyness=0.05, expiry=1.8y: 0.2566


## 6. Visualize the Calibrated Surface

Finally, let's visualize the result. We will create a 3D plot showing the calibrated SVI surface and overlay the original market data points (our synthetic options) to see how well the model fits.

In [6]:
# Create a grid of moneyness and expiry points for the surface plot
moneyness_grid = np.linspace(-0.5, 0.5, 50)
expiry_grid = np.linspace(1.0, 2.0, 25)
X, Y = np.meshgrid(moneyness_grid, expiry_grid)
Z = np.array([surface.volatility(m, e) for m, e in zip(X.ravel(), Y.ravel())]).reshape(X.shape)

# Create the surface plot
surface_plot = go.Surface(x=X, y=Y, z=Z, colorscale='Viridis', opacity=0.8, name='Calibrated SVI Surface')

# Prepare market data for scatter plot
market_data = []
for opt in all_options:
    try:
        vol = implied_volatility_py(opt, opt.price)
        market_data.append({'moneyness': opt.moneyness(), 'expiry': opt.expiry, 'market_vol': vol})
    except CalibrationError:
        pass # Ignore options where implied vol fails
df = pd.DataFrame(market_data)

scatter_plot = go.Scatter3d(
    x=df['moneyness'], y=df['expiry'], z=df['market_vol'],
    mode='markers',
    marker=dict(size=4, color='red', symbol='circle'),
    name='Market Data'
)

fig = go.Figure(data=[surface_plot, scatter_plot])
fig.update_layout(
    title='Calibrated SVI Volatility Surface vs. Market Data',
    scene=dict(
        xaxis_title='Log-Forward Moneyness (k)',
        yaxis_title='Expiry (T)',
        zaxis_title='Implied Volatility'
    ),
    width=900, height=700
)
fig.show()